In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

month=1
year=2023
path = Path("..")/"data"/"processed"/f"ts_data_{year}_{month:02}.parquet"

table = pq.read_table(path)
ts_data = table.to_pandas()
ts_data.head()


,pickup_hour,pickup_location_id,rides,LocationID,Zone
0,2023-01-01 00:00:00,2,0,2,Jamaica Bay
1,2023-01-01 01:00:00,2,0,2,Jamaica Bay
2,2023-01-01 02:00:00,2,0,2,Jamaica Bay
3,2023-01-01 03:00:00,2,0,2,Jamaica Bay
4,2023-01-01 04:00:00,2,0,2,Jamaica Bay


In [3]:
ts_data[ts_data["pickup_location_id"]==43]

,pickup_hour,pickup_location_id,rides,LocationID,Zone
29760,2023-01-01 00:00:00,43,92,43,Central Park
29761,2023-01-01 01:00:00,43,81,43,Central Park
29762,2023-01-01 02:00:00,43,29,43,Central Park
29763,2023-01-01 03:00:00,43,15,43,Central Park
29764,2023-01-01 04:00:00,43,4,43,Central Park
...,...,...,...,...,...
30499,2023-01-31 19:00:00,43,80,43,Central Park
30500,2023-01-31 20:00:00,43,49,43,Central Park
30501,2023-01-31 21:00:00,43,43,43,Central Park
30502,2023-01-31 22:00:00,43,34,43,Central Park


In [6]:
def transform_time_series_to_tabular(df,location_id,feature_col="rides",window_size=12,step_size=1):

    location_data=df[df["pickup_location_id"]==location_id].reset_index(drop=True)

    values = location_data[feature_col].values

    n=len(values)

    if n<=window_size:
        raise ValueError("Not enough data to create even one window of features and target.")
    
    rows=[]

    for i in range(0,n-window_size,step_size):
        features = values[i:i+window_size]
        target = values[i+window_size]
        rows.append(np.append(features,target))

    column_names = [f"feature_{i+1}" for i in range(window_size)] + ["target"]
    transformed_df = pd.DataFrame(rows, columns=column_names)

    return transformed_df

    
    

In [7]:
features_targets = transform_time_series_to_tabular(ts_data,43,"rides",24,1)

In [8]:
features_targets

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,target
0,92,81,29,15,4,4,3,12,12,23,...,119,103,65,39,35,32,40,18,13,2
1,81,29,15,4,4,3,12,12,23,37,...,103,65,39,35,32,40,18,13,2,0
2,29,15,4,4,3,12,12,23,37,41,...,65,39,35,32,40,18,13,2,0,2
3,15,4,4,3,12,12,23,37,41,102,...,39,35,32,40,18,13,2,0,2,2
4,4,4,3,12,12,23,37,41,102,97,...,35,32,40,18,13,2,0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,74,50,33,16,16,3,1,0,1,1,...,97,106,107,109,96,103,156,106,88,80
716,50,33,16,16,3,1,0,1,1,2,...,106,107,109,96,103,156,106,88,80,49
717,33,16,16,3,1,0,1,1,2,16,...,107,109,96,103,156,106,88,80,49,43
718,16,16,3,1,0,1,1,2,16,69,...,109,96,103,156,106,88,80,49,43,34


In [16]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()

In [17]:
print(data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

:Number of Instances: 569

:Number of Attributes: 30 numeric, predictive attributes and the class

:Attribute Information:
    - radius (mean of distances from center to points on the perimeter)
    - texture (standard deviation of gray-scale values)
    - perimeter
    - area
    - smoothness (local variation in radius lengths)
    - compactness (perimeter^2 / area - 1.0)
    - concavity (severity of concave portions of the contour)
    - concave points (number of concave portions of the contour)
    - symmetry
    - fractal dimension ("coastline approximation" - 1)

    The mean, standard error, and "worst" or largest (mean of the three
    worst/largest values) of these features were computed for each image,
    resulting in 30 features.  For instance, field 0 is Mean Radius, field
    10 is Radius SE, field 20 is Worst Radius.

    - 

In [18]:
data.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [19]:
df=pd.DataFrame(data.data,columns=data.feature_names)

In [20]:
df["target"]=data.target

In [21]:
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0
